# Objective: 
- To comprehensively evaluate the performance and robustness of your trading strategy, identifying strengths, weaknesses, and areas for improvement. This involves analyzing both deterministic and Monte Carlo backtest results.

# Load Backtest Results

In [ ]:
import pandas as pd
import os

def load_deterministic_backtest_results(reports_dir="results/backtest_reports"):
    """
    Load deterministic equity curve, trade log, and position log from the specified directory.
    """
    equity_curve = pd.read_csv(os.path.join(reports_dir, "equity_curve.csv"))
    trade_log = pd.read_csv(os.path.join(reports_dir, "trade_log.csv"))
    position_log = pd.read_csv(os.path.join(reports_dir, "position_log.csv"))
    return equity_curve, trade_log, position_log

def load_monte_carlo_results(mc_dir="results/monte_carlo_sims"):
    """
    Load aggregated Monte Carlo metrics and all equity curves from the specified directory.
    """
    metrics_df = pd.read_csv(os.path.join(mc_dir, "monte_carlo_metrics.csv"))
    # Load all equity curves
    equity_curves = []
    for fname in os.listdir(mc_dir):
        if fname.startswith("equity_curve_") and fname.endswith(".csv"):
            eq_curve = pd.read_csv(os.path.join(mc_dir, fname))
            equity_curves.append(eq_curve)
    return metrics_df, equity_curves

# Example usage:
# equity_curve, trade_log, position_log = load_deterministic_backtest_results()
# mc_metrics,

# Deterministic Performance Metrics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

def calculate_performance_metrics(equity_curve, freq='D', risk_free_rate=0.0):
    """
    Calculate comprehensive performance metrics for a deterministic backtest.
    equity_curve: pd.Series or DataFrame with 'portfolio_value' and 'date' columns.
    freq: 'D' for daily, 'W' for weekly, etc.
    risk_free_rate: annual risk-free rate (as decimal).
    Returns: dict of metrics.
    """
    if isinstance(equity_curve, pd.DataFrame):
        if 'portfolio_value' in equity_curve.columns:
            values = equity_curve['portfolio_value'].values
        else:
            raise ValueError("equity_curve DataFrame must have 'portfolio_value' column.")
        if 'date' in equity_curve.columns:
            dates = pd.to_datetime(equity_curve['date'])
        else:
            dates = pd.RangeIndex(len(values))
    else:
        values = equity_curve.values
        dates = equity_curve.index

    returns = np.diff(values) / values[:-1]
    returns = np.insert(returns, 0, 0)  # First return is zero

    # Time calculations
    n_years = (dates[-1] - dates[0]).days / 365.25 if hasattr(dates, '__getitem__') else len(values) / 252
    periods_per_year = {'D': 252, 'W': 52, 'M': 12}.get(freq, 252)

    # Return metrics
    total_return = values[-1] / values[0] - 1
    cagr = (values[-1] / values[0]) ** (1 / n_years) - 1 if n_years > 0 else np.nan
    ann_return = np.mean(returns) * periods_per_year

    # Risk metrics
    ann_vol = np.std(returns) * np.sqrt(periods_per_year)
    max_dd = np.min((values - np.maximum.accumulate(values)) / np.maximum.accumulate(values))
    skewness = skew(returns)
    kurt = kurtosis(returns)

    # Risk-adjusted
    sharpe = (np.mean(returns) - risk_free_rate / periods_per_year) / (np.std(returns) + 1e-8) * np.sqrt(periods_per_year)
    downside_returns = returns[returns < 0]
    sortino = (np.mean(returns) - risk_free_rate / periods_per_year) / (np.std(downside_returns) + 1e-8) * np.sqrt(periods_per_year)
    calmar = cagr / abs(max_dd) if max_dd != 0 else np.nan

    # Trade-level metrics (if available)
    win_rate = profit_factor = avg_win = avg_loss = np.nan
    if 'trade_log' in equity_curve.columns:
        trade_log = equity_curve['trade_log']
        wins = trade_log[trade_log['pnl'] > 0]
        losses = trade_log[trade_log['pnl'] < 0]
        win_rate = len(wins) / (len(wins) + len(losses)) if (len(wins) + len(losses)) > 0 else np.nan
        profit_factor = wins['pnl'].sum() / abs(losses['pnl'].sum()) if losses['pnl'].sum() != 0 else np.nan
        avg_win = wins['pnl'].mean() if not wins.empty else np.nan
        avg_loss = losses['pnl'].mean() if not losses.empty else np.nan

    metrics = {
        "Total Return": total_return,
        "CAGR": cagr,
        "Annualized Return": ann_return,
        "Annualized Volatility": ann_vol,
        "Max Drawdown": max_dd,
        "Sharpe Ratio": sharpe,
        "Sortino Ratio": sortino,
        "Calmar Ratio": calmar,
        "Skewness": skewness,
        "Kurtosis": kurt,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Average Win": avg_win,
        "Average Loss": avg_loss
    }
    return metrics, returns, dates

def plot_equity_curve(equity_curve):
    plt.figure(figsize=(10, 5))
    if isinstance(equity_curve, pd.DataFrame) and 'date' in equity_curve.columns:
        plt.plot(pd.to_datetime(equity_curve['date']), equity_curve['portfolio_value'])
    else:
        plt.plot(equity_curve)
    plt.title("Equity Curve")
    plt.xlabel("Date")
    plt.ylabel("Portfolio Value")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_pnl_histogram(returns, freq='D'):
    plt.figure(figsize=(7, 4))
    plt.hist(returns, bins=50, alpha=0.7)
    plt.title(f"{freq}-level PnL Histogram")
    plt.xlabel("Return")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()

def plot_drawdown(equity_curve):
    if isinstance(equity_curve, pd.DataFrame) and 'portfolio_value' in equity_curve.columns:
        values = equity_curve['portfolio_value'].values
        dates = pd.to_datetime(equity_curve['date'])
    else:
        values = equity_curve.values
        dates = equity_curve.index
    peak = np.maximum.accumulate(values)
    drawdown = (values - peak) / peak
    plt.figure(figsize=(10, 4))
    plt.plot(dates, drawdown, color='red')
    plt.title("Drawdown Chart")
    plt.xlabel("Date")
    plt.ylabel("Drawdown")
    plt.tight_layout()
    plt.show()

# Example usage:
if __name__ == "__main__":
    equity_curve = pd.read_csv("results/backtest_reports/equity_curve.csv")
    metrics, returns, dates = calculate_performance_metrics(equity_curve)
    print("Performance Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")
    plot_equity_curve(equity_curve)
    plot_pnl_histogram(returns)
    plot_drawdown(equity_curve)

# Monte Carlo Performance Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_mc_metrics(metrics_df):
    """
    Analyze and visualize the distribution of Sharpe, Max Drawdown, and Total Return across Monte Carlo paths.
    """
    # Histograms
    plt.figure(figsize=(15, 4))
    for i, metric in enumerate(['Sharpe Ratio', 'Max Drawdown', 'Total Return']):
        plt.subplot(1, 3, i+1)
        sns.histplot(metrics_df[metric], bins=40, kde=True)
        plt.title(f"Distribution of {metric}")
    plt.tight_layout()
    plt.show()

    # Boxplots
    plt.figure(figsize=(10, 4))
    sns.boxplot(data=metrics_df[['Sharpe Ratio', 'Max Drawdown', 'Total Return']])
    plt.title("Boxplot of Key Metrics")
    plt.tight_layout()
    plt.show()

    # Identify worst-case scenarios
    worst_idx = metrics_df['Sharpe Ratio'].idxmin()
    worst_metrics = metrics_df.loc[worst_idx]
    print("Worst-case scenario (lowest Sharpe):")
    print(worst_metrics)

    # Return indices for further analysis
    return worst_idx

def plot_mc_equity_curves(equity_curves, metrics_df, num_curves=100):
    """
    Plot average, median, 10th, and 90th percentile equity curves from Monte Carlo simulations.
    """
    # Align all curves by length
    min_len = min(len(eq['portfolio_value']) for eq in equity_curves)
    aligned_curves = np.array([eq['portfolio_value'][:min_len] for eq in equity_curves])

    avg_curve = np.mean(aligned_curves, axis=0)
    median_curve = np.median(aligned_curves, axis=0)
    p10_curve = np.percentile(aligned_curves, 10, axis=0)
    p90_curve = np.percentile(aligned_curves, 90, axis=0)

    plt.figure(figsize=(12, 6))
    plt.plot(avg_curve, label='Average')
    plt.plot(median_curve, label='Median')
    plt.plot(p10_curve, label='10th Percentile', linestyle='--')
    plt.plot(p90_curve, label='90th Percentile', linestyle='--')
    # Optionally plot a sample of individual paths
    for i in np.random.choice(len(aligned_curves), min(num_curves, len(aligned_curves)), replace=False):
        plt.plot(aligned_curves[i], color='gray', alpha=0.1)
    plt.title("Monte Carlo Equity Curves")
    plt.xlabel("Time Step")
    plt.ylabel("Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Load Monte Carlo results
    metrics_df = pd.read_csv("results/monte_carlo_sims/monte_carlo_metrics.csv")
    equity_curves = []
    import os
    eq_dir = "results/monte_carlo_sims"
    for fname in os.listdir(eq_dir):
        if fname.startswith("equity_curve_") and fname.endswith(".csv"):
            eq = pd.read_csv(os.path.join(eq_dir, fname))
            equity_curves.append(eq)
    # Analyze metrics
    worst_idx = analyze_mc_metrics(metrics_df)
    # Plot equity curves
    plot_mc_equity_curves(equity_curves, metrics_df)
    # Optionally, analyze worst-case path
    print("\nWorst-case equity curve (lowest Sharpe):")
    print(equity_curves[worst_idx].head())

# Strategy Deep Dive & Attribution

In [ ]:
import pandas as pd
import numpy as np

def analyze_trade_stats(trade_log):
    """
    Analyze trade frequency, average holding period, and PnL by trade type.
    Assumes trade_log has columns: ['date', 'ticker', 'side', 'size', 'fill_price', 'slippage', 'pnl', 'event_type']
    """
    trade_log['date'] = pd.to_datetime(trade_log['date'])
    # Trade frequency
    freq = trade_log.groupby('side').size().to_dict()
    # Average holding period (if entry/exit pairs are available)
    holding_periods = []
    trade_log = trade_log.sort_values(['ticker', 'date'])
    for ticker, group in trade_log.groupby('ticker'):
        entries = group[group['side'] == 'buy']
        exits = group[group['side'] == 'sell']
        for _, entry in entries.iterrows():
            exit = exits[exits['date'] > entry['date']].head(1)
            if not exit.empty:
                holding_periods.append((exit['date'].values[0] - entry['date']).astype('timedelta64[D]').astype(int))
    avg_holding = np.mean(holding_periods) if holding_periods else np.nan
    # PnL by trade type
    pnl_by_side = trade_log.groupby('side')['pnl'].sum().to_dict()
    pnl_by_event = trade_log.groupby('event_type')['pnl'].sum().to_dict() if 'event_type' in trade_log.columns else {}

    print("Trade Frequency by Side:", freq)
    print("Average Holding Period (days):", avg_holding)
    print("PnL by Side:", pnl_by_side)
    print("PnL by Event Type:", pnl_by_event)
    return freq, avg_holding, pnl_by_side, pnl_by_event

def feature_attribution(trade_log, features_df, feature_cols, pnl_col='pnl', regime_col=None):
    """
    Rudimentary attribution: Which features contributed most to profitable trades?
    Optionally, analyze by market regime.
    """
    # Merge features at trade entry with trade log
    merged = pd.merge(trade_log, features_df, on=['date', 'ticker'], how='left')
    # Correlation of features with PnL
    corrs = merged[feature_cols + [pnl_col]].corr()[pnl_col].drop(pnl_col)
    print("Feature-PnL Correlations:")
    print(corrs.sort_values(ascending=False))
    # Attribution by regime
    if regime_col and regime_col in merged.columns:
        for regime, group in merged.groupby(regime_col):
            print(f"\nRegime: {regime}")
            rcorrs = group[feature_cols + [pnl_col]].corr()[pnl_col].drop(pnl_col)
            print(rcorrs.sort_values(ascending=False))
    return corrs

if __name__ == "__main__":
    # Load logs and features
    trade_log = pd.read_csv("results/backtest_reports/trade_log.csv")
    features_df = pd.read_csv("data/features/AAPL_ohlcv_features.csv")  # Example, adjust as needed
    feature_cols = [col for col in features_df.columns if "momentum" in col or "volatility" in col or "volume" in col]
    # Analyze trade stats
    analyze_trade_stats(trade_log)
    # Attribution
    feature_attribution(trade_log, features_df, feature_cols)

# Sensitivity Analysis & Stress Testing

In [ ]:
costs = [0.0001, 0.0005, 0.001, 0.002]
thresholds = [0.005, 0.01, 0.02]
results = []
for cost in costs:
    for thresh in thresholds:
        # Update config
        config['transaction_cost'] = cost
        config['signal_threshold'] = thresh
        # Run backtest (script or function)
        metrics = run_backtest_with_config(config)
        results.append({'cost': cost, 'threshold': thresh, **metrics})
df = pd.DataFrame(results)
# Plot sensitivity

# Restrict to 2008-2009
crisis_data = market_data[(market_data['date'] >= '2008-01-01') & (market_data['date'] <= '2009-12-31')]
metrics = run_backtest_with_data(crisis_data, config)
print(metrics)

# Simulate flash crash
shock_data = market_data.copy()
shock_data.loc[shock_data['date'] == '2010-05-06', 'close'] *= 0.85  # 15% drop
metrics = run_backtest_with_data(shock_data, config)
print(metrics)

# Summary & Recommendations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from jinja2 import Template
import datetime
import os

def summarize_findings(deterministic_metrics, mc_metrics_df):
    # Deterministic summary
    det_summary = {
        "Total Return": deterministic_metrics["Total Return"],
        "CAGR": deterministic_metrics["CAGR"],
        "Sharpe Ratio": deterministic_metrics["Sharpe Ratio"],
        "Max Drawdown": deterministic_metrics["Max Drawdown"],
        "Calmar Ratio": deterministic_metrics["Calmar Ratio"]
    }
    # Monte Carlo summary
    mc_summary = {
        "Sharpe Ratio (mean)": mc_metrics_df["Sharpe Ratio"].mean(),
        "Sharpe Ratio (10th pct)": mc_metrics_df["Sharpe Ratio"].quantile(0.1),
        "Sharpe Ratio (90th pct)": mc_metrics_df["Sharpe Ratio"].quantile(0.9),
        "Max Drawdown (mean)": mc_metrics_df["Max Drawdown"].mean(),
        "Total Return (mean)": mc_metrics_df["Total Return"].mean()
    }
    # Strengths & weaknesses
    strengths = []
    weaknesses = []
    if det_summary["Sharpe Ratio"] > 1 and mc_summary["Sharpe Ratio (10th pct)"] > 0.5:
        strengths.append("Consistent risk-adjusted returns across most scenarios.")
    if det_summary["Max Drawdown"] > -0.2:
        strengths.append("Controlled drawdowns in deterministic and most MC paths.")
    if mc_metrics_df["Sharpe Ratio"].min() < 0:
        weaknesses.append("Some Monte Carlo paths show negative Sharpe ratios (strategy can underperform in adverse conditions).")
    if det_summary["Calmar Ratio"] < 1:
        weaknesses.append("Calmar Ratio below 1 indicates risk of large drawdowns relative to return.")
    if mc_metrics_df["Max Drawdown"].min() < -0.5:
        weaknesses.append("Severe drawdowns possible in worst-case scenarios.")

    # Future research
    future = [
        "Engineer new features (e.g., alternative data, regime indicators).",
        "Experiment with advanced ML models (transformers, ensemble stacking).",
        "Enhance risk controls (dynamic position sizing, stop-loss logic).",
        "Improve execution simulation (market impact, order book modeling)."
    ]
    return det_summary, mc_summary, strengths, weaknesses, future

def generate_html_report(det_summary, mc_summary, strengths, weaknesses, future, output_path="results/final_performance_report.html"):
    template = Template("""
    <html>
    <head><title>Final Performance Report</title></head>
    <body>
    <h1>Strategy Performance Report</h1>
    <h2>1. Deterministic Backtest Summary</h2>
    <ul>
      {% for k, v in det_summary.items() %}
        <li><b>{{k}}:</b> {{'%0.4f' % v if v is not None else 'N/A'}}</li>
      {% endfor %}
    </ul>
    <h2>2. Monte Carlo Simulation Summary</h2>
    <ul>
      {% for k, v in mc_summary.items() %}
        <li><b>{{k}}:</b> {{'%0.4f' % v if v is not None else 'N/A'}}</li>
      {% endfor %}
    </ul>
    <h2>3. Strengths</h2>
    <ul>
      {% for s in strengths %}
        <li>{{s}}</li>
      {% endfor %}
    </ul>
    <h2>4. Weaknesses</h2>
    <ul>
      {% for w in weaknesses %}
        <li>{{w}}</li>
      {% endfor %}
    </ul>
    <h2>5. Areas for Future Research & Improvement</h2>
    <ul>
      {% for f in future %}
        <li>{{f}}</li>
      {% endfor %}
    </ul>
    <p><i>Report generated on {{date}}</i></p>
    </body>
    </html>
    """)
    html = template.render(
        det_summary=det_summary,
        mc_summary=mc_summary,
        strengths=strengths,
        weaknesses=weaknesses,
        future=future,
        date=datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    )
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as f:
        f.write(html)
    print(f"Performance report saved to {output_path}")

if __name__ == "__main__":
    # Load deterministic and MC metrics
    det_metrics = pd.read_csv("results/backtest_reports/deterministic_metrics.csv").iloc[0].to_dict()
    mc_metrics_df = pd.read_csv("results/monte_carlo_sims/monte_carlo_metrics.csv")
    det_summary, mc_summary, strengths, weaknesses, future = summarize_findings(det_metrics, mc_metrics_df)
    generate_html_report(det_summary, mc_summary, strengths, weaknesses, future)